# **Assignment 2**
## Title : Predicting breast cancer whether malignant or benign using SVM
### Nikhil Bokade
### CMS1902

In [4]:
import pandas as pd
import numpy as np

### **Task 1: Read Data and Split into train and test (80-20)**

In [5]:
# Loading data 
df = pd.read_csv('drive/My Drive/CMS/ML2/Breast_cancer.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [6]:
id = df['id']
df = df.drop(['id','Unnamed: 32'],axis=1)
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [25]:
X = df.drop('diagnosis',axis=1)
y = df['diagnosis']

In [8]:
X.shape, y.shape

((569, 30), (569,))

In [9]:
df['diagnosis'].value_counts()

B    357
M    212
Name: diagnosis, dtype: int64

In [26]:
# Converting target into binary class
y = y.map({'M':1,'B':0})

In [27]:
# Splitting data into train and test set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    stratify=y, 
                                                    random_state=42)        
                                                    

In [12]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((455, 30), (114, 30), (455,), (114,))

### **Task 2 : Finding best parameters using GridSearch**

In [13]:
# Libraries for metrics
from sklearn.metrics import accuracy_score, matthews_corrcoef, confusion_matrix, make_scorer

# SVM model
from sklearn.svm import SVC 

In [14]:
# Grid Serach to find best parameters
from sklearn.model_selection import GridSearchCV, cross_val_score

scorer = make_scorer(matthews_corrcoef)

param_grid = {'C':[1,10,20,50,100],
              'gamma': [1, 0.1, 0.01, 0.001],
              'kernel': ['rbf', 'linear']}

grid = GridSearchCV(SVC(),param_grid, cv=5, verbose=1,scoring=scorer)
grid.fit(X_train,y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_y

GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 10, 20, 50, 100],
                         'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf', 'linear']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=make_scorer(matthews_corrcoef), verbose=1)

In [15]:
best_param = grid.best_params_
print(best_param)

{'C': 50, 'gamma': 1, 'kernel': 'linear'}


### **Task 3: With best parameters, fit model and find accuracy and mcc on test data**

In [16]:
model = SVC(C = 50, gamma = 1, kernel = 'linear')
model.fit(X_train, y_train)
y_preds = model.predict(X_test)
acc = accuracy_score(y_test, y_preds)
print('Accuracy',acc)
mcc = matthews_corrcoef(y_test, y_preds)
print('MCC',mcc)

Accuracy 0.9385964912280702
MCC 0.8674927968252835


### **Task 4: Divide the data into 5 folds using numpy and find mean CV accuracy and mean CV MCC**

In [30]:
fold = 5
shuf = np.random.choice(fold, len(X))
Accuracy = []
MCC = []
for i in range(5):
  X_train = X.iloc[np.where(shuf != i)[0],]
  X_test = X.iloc[np.where(shuf == i)[0],]
  y_train = y.iloc[np.where(shuf != i)[0]]
  y_test = y.iloc[np.where(shuf == i)[0]]

  
  model = SVC(C = 50, gamma = 1, kernel = 'linear')
  model.fit(X_train, y_train)

  y_pred = model.predict(X_test)
  acc = accuracy_score(y_test, y_pred)
  print("Accuracy : ",acc)
  Accuracy.append(acc)
  mcc = matthews_corrcoef(y_test, y_pred)
  print("MCC:",mcc)
  MCC.append(mcc)

Accuracy = np.array(Accuracy)
MCC = np.array(MCC)
mean_cv_acc = Accuracy.mean()
print("Mean CV Accuracy: ",mean_cv_acc)
mean_mcc = MCC.mean()
print("Mean MCC: ",mean_mcc)

Accuracy :  0.9809523809523809
MCC: 0.9616294415187687
Accuracy :  0.9534883720930233
MCC: 0.9012755102040816
Accuracy :  0.9629629629629629
MCC: 0.922077922077922
Accuracy :  0.9646017699115044
MCC: 0.9262564300613756
Accuracy :  0.956140350877193
MCC: 0.9009739556258517
Mean CV Accuracy:  0.9636291673594128
Mean MCC:  0.9224426518975999


### **Task 5: Use sklearn cross_val score to find mean cv acc and mean cv mcc**

In [22]:
score_acc = cross_val_score(SVC(),X,y,cv=5)
print("Accuracy: ",score_acc)
print("Mean of CV Accuracy: ",score_acc.mean())
mcc_score = cross_val_score(SVC(),X, y,cv=5, scoring = scorer)
print("MCC: ",mcc_score)
print("Mean of CV MCC: ",mcc_score.mean())

Accuracy:  [0.85087719 0.89473684 0.92982456 0.94736842 0.9380531 ]
Mean of CV Accuracy:  0.9121720229777983
MCC:  [0.69115896 0.77682034 0.85037769 0.88724426 0.86685085]
Mean of CV MCC:  0.8144904196922459
